<a href="https://colab.research.google.com/drive/1ZQG9Hnv1caDkcsmoCAf8_ynpqs3IQmST?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

### Self-Refine: Iterative Refinement with Self-Feedback

In [1]:
!pip install -qU google-generativeai

In [2]:
import google.generativeai as genai
import getpass

Get free-tier Google's Gemini API Key here: https://aistudio.google.com/app/apikey

In [8]:
API_KEY = getpass.getpass("Enter your Google API key: ")

Enter your Google API key: ··········


In [11]:
genai.configure(api_key=API_KEY)

In [12]:
class SelfRefineAgent:
    def __init__(self):
        self.model = genai.GenerativeModel("gemini-2.0-flash-exp")
        self.history = []

    def generate(self, task):
        """Generate initial output"""
        prompt = f"""Generate a response for this task:

        Task: {task}

        Response:"""

        response = self.model.generate_content(prompt).text
        return response.strip()

    def critique(self, task, output):
        """Self-critique: Identify issues and areas for improvement"""
        prompt = f"""Task: {task}

        Current Output:
        {output}

        Critique this output. Identify:
        1. What's good about it
        2. What problems or errors exist
        3. Specific improvements needed

        Critique:"""

        response = self.model.generate_content(prompt).text
        return response.strip()

    def refine(self, task, output, critique):
        """Refine output based on critique"""
        prompt = f"""Task: {task}

        Current Output:
        {output}

        Critique:
        {critique}

        Based on the critique, generate an improved version:"""

        response = self.model.generate_content(prompt).text
        return response.strip()

    def evaluate_quality(self, task, output):
        """Evaluate if output meets quality standards (0-10)"""
        prompt = f"""Task: {task}

        Output:
        {output}

        Rate the quality of this output (0-10) based on:
        - Correctness
        - Completeness
        - Clarity
        - Relevance

        Score (just number):"""

        response = self.model.generate_content(prompt).text

        try:
            score = float(response.strip().split()[0])
            return min(max(score, 0), 10)
        except:
            return 5.0

    def self_refine(self, task, max_iterations=5, quality_threshold=8.0):
        """Main self-refinement loop"""
        print(f"\n{'='*60}")
        print(f"🔄 Self-Refine Loop")
        print(f"{'='*60}")
        print(f"Task: {task}\n")

        # Step 1: Generate initial output
        print(f"{'─'*60}")
        print(f"ITERATION 1: Initial Generation")
        print(f"{'─'*60}\n")

        output = self.generate(task)
        quality = self.evaluate_quality(task, output)

        print(f"Generated Output:\n{output}\n")
        print(f"Quality Score: {quality}/10\n")

        self.history.append({
            "iteration": 1,
            "output": output,
            "quality": quality,
            "critique": None
        })

        # Refinement loop
        for iteration in range(2, max_iterations + 1):
            # Check if quality threshold met
            if quality >= quality_threshold:
                print(f"✅ Quality threshold ({quality_threshold}) reached!")
                break

            print(f"{'─'*60}")
            print(f"ITERATION {iteration}: Critique & Refine")
            print(f"{'─'*60}\n")

            # Step 2: Self-critique
            print("🔍 Self-Critique:")
            critique = self.critique(task, output)
            print(f"{critique}\n")

            # Step 3: Refine based on critique
            print("✨ Refining output...\n")
            output = self.refine(task, output, critique)

            # Step 4: Evaluate new quality
            quality = self.evaluate_quality(task, output)

            print(f"Refined Output:\n{output}\n")
            print(f"Quality Score: {quality}/10\n")

            self.history.append({
                "iteration": iteration,
                "output": output,
                "quality": quality,
                "critique": critique
            })

            # Check for diminishing returns
            if iteration > 2:
                prev_quality = self.history[-2]["quality"]
                improvement = quality - prev_quality
                if improvement < 0.5:
                    print(f"⚠️  Minimal improvement ({improvement:.1f}), stopping.")
                    break

        # Show final result
        print(f"{'='*60}")
        print(f"🏆 FINAL OUTPUT (Iteration {iteration})")
        print(f"{'='*60}")
        print(output)
        print(f"\nFinal Quality: {quality}/10\n")

        self._show_history()

        return output

    def _show_history(self):
        """Display refinement history"""
        print(f"{'='*60}")
        print(f"📊 REFINEMENT HISTORY")
        print(f"{'='*60}")

        for entry in self.history:
            print(f"Iteration {entry['iteration']}: Quality {entry['quality']:.1f}/10")

        if len(self.history) > 1:
            improvement = self.history[-1]["quality"] - self.history[0]["quality"]
            print(f"\nTotal Improvement: +{improvement:.1f} points")
        print()

In [13]:
# Example 1: Code Refinement
print("="*60)
print("EXAMPLE 1: Code Refinement")
print("="*60)

agent1 = SelfRefineAgent()
agent1.self_refine(
    "Write a Python function to find the factorial of a number",
    max_iterations=4,
    quality_threshold=8.5
)


# Example 2: Writing Improvement
print("\n" + "="*60)
print("EXAMPLE 2: Writing Improvement")
print("="*60)

agent2 = SelfRefineAgent()
agent2.self_refine(
    "Write a professional email declining a job offer politely",
    max_iterations=4,
    quality_threshold=8.0
)


# Example 3: Translation Polishing
print("\n" + "="*60)
print("EXAMPLE 3: Translation Polishing")
print("="*60)

agent3 = SelfRefineAgent()
agent3.self_refine(
    "Translate 'The early bird catches the worm' to Spanish, preserving the meaning and cultural relevance",
    max_iterations=3,
    quality_threshold=8.5
)


# Example 4: Algorithm Enhancement
print("\n" + "="*60)
print("EXAMPLE 4: Algorithm Enhancement")
print("="*60)

agent4 = SelfRefineAgent()
agent4.self_refine(
    "Design an algorithm to find the shortest path in a weighted graph",
    max_iterations=4,
    quality_threshold=8.0
)


# Example 5: Content Summarization
print("\n" + "="*60)
print("EXAMPLE 5: Content Summarization")
print("="*60)

agent5 = SelfRefineAgent()
agent5.self_refine(
    "Summarize the key benefits of remote work in 3 concise bullet points",
    max_iterations=3,
    quality_threshold=8.5
)


# Example 6: Creative Writing
print("\n" + "="*60)
print("EXAMPLE 6: Creative Idea Iteration")
print("="*60)

agent6 = SelfRefineAgent()
agent6.self_refine(
    "Create a tagline for an eco-friendly water bottle brand",
    max_iterations=4,
    quality_threshold=8.0
)


print("✅ Self-Refine Complete!")

EXAMPLE 1: Code Refinement

🔄 Self-Refine Loop
Task: Write a Python function to find the factorial of a number

────────────────────────────────────────────────────────────
ITERATION 1: Initial Generation
────────────────────────────────────────────────────────────

Generated Output:
```python
def factorial(n):
  """
  This function calculates the factorial of a non-negative integer.

  Args:
    n: A non-negative integer.

  Returns:
    The factorial of n (n!), or 1 if n is 0.
    Returns None if n is negative.
  """
  if n < 0:
    return None  # Factorial is not defined for negative numbers
  elif n == 0:
    return 1
  else:
    result = 1
    for i in range(1, n + 1):
      result *= i
    return result

# Example Usage:
if __name__ == '__main__':
  print(factorial(5))  # Output: 120
  print(factorial(0))  # Output: 1
  print(factorial(-1)) # Output: None
```

**Explanation:**

1. **Function Definition:**
   - The code defines a function called `factorial(n)` that takes one argum

TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-exp:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota. Please migrate to Gemini 2.0 Flash Preview (Image Generation) (models/gemini-2.0-flash-preview-image-generation) for higher quota limits. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.